# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,5.78,75,35,3.04,GS,1741629241
1,1,punta arenas,-53.1500,-70.9167,8.06,53,75,12.35,CL,1741629077
2,2,santa eularia des riu,38.9846,1.5341,15.01,77,20,2.06,ES,1741629243
3,3,atafu village,-8.5421,-172.5159,27.57,78,28,6.07,TK,1741629244
4,4,albany,42.6001,-73.9662,14.20,34,31,0.89,US,1741629157


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_l = city_data_df["Max Temp"] > 11
ideal_temp_h = city_data_df["Max Temp"] < 21
ideal_speed_w = city_data_df["Wind Speed"] < 5
ideal_humidity = city_data_df["Humidity"] < 75

ideal_weather_df = city_data_df[ideal_temp_l & ideal_temp_h & ideal_speed_w & ideal_humidity]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,albany,42.6001,-73.9662,14.20,34,31,0.89,US,1741629157
5,5,bethel,41.3712,-73.4140,16.70,28,0,4.63,US,1741629246
16,16,hermanus,-34.4187,19.2345,18.22,68,26,3.50,ZA,1741629055
39,39,blackmans bay,-43.0167,147.3167,13.18,63,78,2.54,AU,1741629223
41,41,tura,25.5198,90.2201,20.54,32,0,1.86,IN,1741629291


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
4,albany,US,42.6001,-73.9662,34,
5,bethel,US,41.3712,-73.4140,28,
16,hermanus,ZA,-34.4187,19.2345,68,
39,blackmans bay,AU,-43.0167,147.3167,63,
41,tura,IN,25.5198,90.2201,32,
48,lompoc,US,34.6391,-120.4579,61,
58,benghazi,LY,32.1167,20.0667,54,
67,pocahontas,US,36.2615,-90.9712,33,
75,guerrero negro,MX,27.9769,-114.0611,43,
76,chonchi,CL,-42.6128,-73.8092,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
hermanus - nearest hotel: Aloe guest house
blackmans bay - nearest hotel: Villa Howden
tura - nearest hotel: Hotel Polo Orchid
lompoc - nearest hotel: Red Roof Inn Lompoc
benghazi - nearest hotel: ترجاب
pocahontas - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza Salparaíso
chonchi - nearest hotel: Hotel Huildín
jinghong - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
papatowai - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
jamestown - nearest hotel: DoubleTree Jamestown
weiser - nearest hotel: State Street Motel
ardestan - nearest hotel: Tourism Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
colonia - nearest hotel: Wasserturm Hotel Cologne
hamilton - nearest hotel: North Vista Manor
enoch - nearest hotel: T

,City,Country,Lat,Lng,Humidity,Hotel Name
4,albany,US,42.6001,-73.9662,34,No hotel found
5,bethel,US,41.3712,-73.4140,28,Hampton Inn Danbury
16,hermanus,ZA,-34.4187,19.2345,68,Aloe guest house
39,blackmans bay,AU,-43.0167,147.3167,63,Villa Howden
41,tura,IN,25.5198,90.2201,32,Hotel Polo Orchid
48,lompoc,US,34.6391,-120.4579,61,Red Roof Inn Lompoc
58,benghazi,LY,32.1167,20.0667,54,ترجاب
67,pocahontas,US,36.2615,-90.9712,33,No hotel found
75,guerrero negro,MX,27.9769,-114.0611,43,Plaza Salparaíso
76,chonchi,CL,-42.6128,-73.8092,42,Hotel Huildín


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)